In [2]:
import utils.datasets

# import util.datasets
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

2022-10-17 19:03:04.686092: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-17 19:03:04.821362: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-17 19:03:04.827619: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-17 19:03:04.827635: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [ ]:
train, test = utils.datasets.pascal_voc_detection(os.path.normpath("/home/FYP/wang1570/datasets/pascal-voc-2012/VOCdevkit/VOC2012")).load()

loaded using cache file


In [ ]:
from easydict import EasyDict as edict
import numpy as np
import tensorflow as tf

# pascal voc 2012
pascal_voc_2012 = edict()
pascal_voc_2012.classes = ['__background__', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat',
                    'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant',
                    'sheep', 'sofa', 'train', 'tvmonitor']
pascal_voc_2012.pixel_means = np.array([[[102.9801, 115.9465, 122.7717]]])
pascal_voc_2012.max_size = 1000
pascal_voc_2012.min_size = 600

# vgg16
vgg16 = edict()
vgg16.feature_stride = 16 # when input size = 224, output = 14, strive = 224/14

# anchor
anchors = edict()
anchors.ratios = np.array([0.5, 1, 2])
anchors.scales = np.array([8, 16, 32])
anchors.num = len(anchors.ratios) * len(anchors.scales) # 1 anchor for each combination

# region proposal network
rpn = edict()
rpn.rpn_chan = 512

# arch
frcnn = edict()
frcnn.use_bias = False
frcnn.regulizer = None
frcnn.initializer = tf.keras.initializers.RandomNormal(stddev=0.01)
frcnn.backbone = tf.keras.applications.vgg16.VGG16(False, None)
frcnn.nms = edict()
frcnn.nms.output_size = 2000
frcnn.nms.iou = 0.7

# overall
cfg = edict()
cfg.db = pascal_voc_2012
cfg.backbone = vgg16
cfg.anchors = anchors
cfg.frcnn = frcnn


2022-10-14 16:29:56.040857: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-14 16:29:56.040898: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-14 16:29:56.040941: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (SCSEGPU-TC1): /proc/driver/nvidia/version does not exist
2022-10-14 16:29:56.041320: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
class proposal_layer(tf.keras.layers.Layer):
  def __init__(self, name = "proposal_layer"):
    super().__init__(name = "proposal_layer")
    self.rpn_cls_score_conv = tf.keras.layers.Conv2D(cfg.anchors.num * 2, kernel_size=(1, 1), padding='same', use_bias=cfg.frcnn.use_bias, kernel_initializer=cfg.frcnn.initializer, kernel_regularizer=cfg.frcnn.regulizer)
    self.rpn_cls_prob_softmax = tf.keras.layers.Softmax()
    self.rpn_bbox_pred_conv = tf.keras.layers.Conv2D(cfg.anchors.num * 4,kernel_size=(1, 1), padding='same',  use_bias=cfg.frcnn.use_bias, kernel_initializer=cfg.frcnn.initializer, kernel_regularizer=cfg.frcnn.regulizer)

  def call(self, rpn_features, anchors, img_sz):
    rpn_cls_score = self.rpn_cls_score_conv(rpn_features)
    rpn_cls_score_reshape = tf.reshape(rpn_cls_score, (-1, 2))
    rpn_cls_prob = self.rpn_cls_prob_softmax(rpn_cls_score_reshape)
    rpn_cls_pred = tf.argmax(rpn_cls_prob, axis = -1)
    rpn_cls_prob = tf.reshape(rpn_cls_prob, tf.shape(rpn_cls_score))

    rpn_bbox_pred = self.rpn_bbox_pred_conv(rpn_features)
    rpn_bbox_pred = tf.reshape(rpn_bbox_pred, (-1, 4))

    rpn_cls_prob_fgrd = rpn_cls_prob[:, :, :, cfg.anchors.num:]
    rpn_cls_prob_fgrd = tf.reshape(rpn_cls_prob_fgrd, (-1, ))

    anchor_preds = self.convert_and_clip(anchors, rpn_bbox_pred, img_sz)

    proposal_ids = tf.image.non_max_suppression(anchor_preds, rpn_cls_prob_fgrd, max_output_size=cfg.frcnn.nms.output_size, iou_threshold=cfg.frcnn.nms.iou)

    proposals = anchor_preds[proposal_ids]
    proposal_scores = rpn_cls_prob_fgrd[proposal_ids]

    return proposals, proposal_scores

In [ ]:
a = proposal_layer()

In [ ]:
a.name

'proposal_layer'

In [ ]:
a = tf.range(50)

In [ ]:
a = tf.reshape(a, (5, 5, 2))

In [ ]:
a = tf.random.uniform((5, 5, 2))

In [ ]:
b = tf.reshape(a, (-1, 2))
b = tf.cast(b, dtype = 'float')
b

<tf.Tensor: shape=(25, 2), dtype=float32, numpy=
array([[0.717497  , 0.04890013],
       [0.16746831, 0.626809  ],
       [0.40473008, 0.82172346],
       [0.60019577, 0.7794467 ],
       [0.17423964, 0.98111546],
       [0.5856674 , 0.45322704],
       [0.3817389 , 0.12977862],
       [0.9745786 , 0.30084693],
       [0.454031  , 0.7016636 ],
       [0.03043282, 0.0470978 ],
       [0.08154368, 0.60092425],
       [0.205073  , 0.49768186],
       [0.6479666 , 0.09068   ],
       [0.28856456, 0.00207877],
       [0.7204088 , 0.03900468],
       [0.80403876, 0.6167313 ],
       [0.81875885, 0.8785411 ],
       [0.96998656, 0.39076102],
       [0.597306  , 0.37829578],
       [0.14391887, 0.1101414 ],
       [0.12568688, 0.9189284 ],
       [0.3242582 , 0.74013686],
       [0.06414032, 0.52793026],
       [0.9763073 , 0.15707576],
       [0.6634358 , 0.40364373]], dtype=float32)>

In [ ]:
sfx = tf.keras.layers.Softmax()

In [ ]:
c = sfx(b)

In [ ]:
b = sfx(b)

In [6]:
def get_area(boxes):
  x0, y0, x1, y1 = tf.split(value=boxes, num_or_size_splits=4, axis=1)
  return tf.reshape((x1 - x0 + 1) * (y1 - y0 + 1), (-1, 1))
  
def get_intersection(boxes1, boxes2):
  x0_1, y0_1, x1_1, y1_1 = tf.split(value=boxes1, num_or_size_splits=4, axis=1)
  x0_2, y0_2, x1_2, y1_2 = tf.split(value=boxes2, num_or_size_splits=4, axis=1)
  max_x0 = tf.maximum(x0_1, tf.transpose(x0_2))
  max_y0 = tf.maximum(y0_1, tf.transpose(y0_2))
  min_x1 = tf.minimum(x1_1, tf.transpose(x1_2))
  min_y1 = tf.minimum(y1_1, tf.transpose(y1_2))
  height = tf.maximum(0, min_x1 - max_x0 + 1)
  width = tf.maximum(0, min_y1 - max_y0 + 1)
  return height * width

def get_overlap(anchors, gt_boxes):
  anchor_area = get_area(anchors)
  gt_boxes_area = get_area(gt_boxes)
  intersection = get_intersection(anchors, gt_boxes)
  union = anchor_area + tf.reshape(gt_boxes_area, (1, -1)) - intersection
  return intersection / union

In [7]:
anchors = tf.Variable( 
  [[1, 1, 2 ,2],
  [0, 0, 1, 1],
  [0, 0, 0, 0],
  [1, 1, 2 ,2],
  [3, 3, 4, 4]],
dtype = float)
gt_boxes = tf.Variable(
  [[1, 1, 2, 2],
  [1, 1, 2, 2],
  [0, 0, 1, 1]]
, dtype = float)
overlaps = get_overlap(anchors, gt_boxes)
labels = tf.zeros(anchors.shape[0], dtype = 'float')
labels -= 1

In [8]:
argmax_overlaps = tf.cast(tf.argmax(overlaps, axis=1), dtype=tf.int32)
argmax_gather_nd_inds = tf.stack([tf.range(tf.shape(overlaps)[0]), argmax_overlaps], axis=1)
max_overlaps = tf.gather_nd(overlaps, argmax_gather_nd_inds)

gt_argmax_overlaps = tf.cast(tf.argmax(overlaps, axis=0), dtype=tf.int32)
max_overlaps_gather_nd_inds = tf.stack([gt_argmax_overlaps, tf.range(tf.shape(overlaps)[1])], axis=1)
gt_max_overlaps = tf.gather_nd(overlaps, max_overlaps_gather_nd_inds)
gt_argmax_overlaps = tf.where(overlaps == gt_max_overlaps)[:, 0]

In [9]:
unique_gt_argmax_overlaps = tf.unique(gt_argmax_overlaps)[0]
highest_fg_label = tf.ones(unique_gt_argmax_overlaps.shape)
highest_gt_row_ids_expand_dim = tf.expand_dims(unique_gt_argmax_overlaps, axis=1)
labels = tf.where(max_overlaps < 0.3, tf.zeros_like(labels, dtype = 'float'), labels)
labels = tf.where(max_overlaps >= 0.7, tf.ones_like(labels, dtype = 'float'), labels)
labels = tf.tensor_scatter_nd_update(labels, highest_gt_row_ids_expand_dim, highest_fg_label)

In [13]:
num_fg = int(0.5 * 4)
fg_inds = tf.reshape(tf.where(labels == 1), shape=(-1,))
fg_inds_num = tf.shape(fg_inds)[0]

In [15]:
fg_inds, fg_inds_num

(<tf.Tensor: shape=(3,), dtype=int64, numpy=array([0, 1, 3])>,
 <tf.Tensor: shape=(), dtype=int32, numpy=3>)

In [16]:
fg_flag = tf.cast(fg_inds_num > num_fg, dtype=tf.float32)

In [17]:
fg_flag

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [27]:
a = tf.Variable([1, 2, 3, 4])

In [28]:
a

<tf.Variable 'Variable:0' shape=(4,) dtype=int32, numpy=array([1, 2, 3, 4], dtype=int32)>

In [29]:
b = a[1 : 2]

In [33]:
c = a * 10

In [34]:
c

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([10, 20, 30, 40], dtype=int32)>

In [35]:
a = tf.random.uniform((10, 4))

In [39]:
x0, y0, x1, y1 = tf.split(a, 4, axis = 1)

In [44]:
x0

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[0.05842113],
       [0.28271675],
       [0.05620921],
       [0.57703745],
       [0.61593354],
       [0.24065948],
       [0.37477183],
       [0.5346147 ],
       [0.71934605],
       [0.86064816]], dtype=float32)>

In [46]:
a = tf.constant([1, 2, 3])

In [47]:
tf.expand_dims(a, axis = 1)

<tf.Tensor: shape=(3, 1), dtype=int32, numpy=
array([[1],
       [2],
       [3]], dtype=int32)>

In [48]:
tf.fill((9, 9), 10)

<tf.Tensor: shape=(9, 9), dtype=int32, numpy=
array([[10, 10, 10, 10, 10, 10, 10, 10, 10],
       [10, 10, 10, 10, 10, 10, 10, 10, 10],
       [10, 10, 10, 10, 10, 10, 10, 10, 10],
       [10, 10, 10, 10, 10, 10, 10, 10, 10],
       [10, 10, 10, 10, 10, 10, 10, 10, 10],
       [10, 10, 10, 10, 10, 10, 10, 10, 10],
       [10, 10, 10, 10, 10, 10, 10, 10, 10],
       [10, 10, 10, 10, 10, 10, 10, 10, 10],
       [10, 10, 10, 10, 10, 10, 10, 10, 10]], dtype=int32)>